In [1]:
import yfinance as yf
import time
import json
from datetime import datetime
from kafka import KafkaProducer

# Kafka setup
producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Stock tickers
tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMD', 'AMZN', 'IBM', 'ORCL', 'ACN', 'INFY', 'SAP']

def fetch_prices():
    data = yf.download(tickers, period="1d", interval="15m", progress=False, group_by='ticker')
    stock_info = {}
    for ticker in tickers:
        try:
            df = data[ticker].dropna()
            latest = df.iloc[-1]
            stock_info[ticker] = {
                'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                'Open': round(latest['Open'], 2),
                'High': round(latest['High'], 2),
                'Low': round(latest['Low'], 2),
                'Close': round(latest['Close'], 2),
                'Volume': int(latest['Volume'])
            }
        except Exception as e:
            stock_info[ticker] = {'error': str(e)}
    return stock_info

# Stream to Kafka every 2 seconds
while True:
    prices = fetch_prices()
    for ticker, data in prices.items():
        message = {'ticker': ticker, 'data': data}
        producer.send('stock_prices', message)
        print(f"Sent: {message}")
    time.sleep(2)


YF.download() has changed argument auto_adjust default to True
Sent: {'ticker': 'AAPL', 'data': {'timestamp': '2025-04-14 09:18:37', 'Open': np.float64(198.1), 'High': np.float64(199.0), 'Low': np.float64(197.58), 'Close': np.float64(198.02), 'Volume': 7267282}}
Sent: {'ticker': 'GOOGL', 'data': {'timestamp': '2025-04-14 09:18:37', 'Open': np.float64(157.06), 'High': np.float64(157.63), 'Low': np.float64(156.79), 'Close': np.float64(157.0), 'Volume': 3428925}}
Sent: {'ticker': 'MSFT', 'data': {'timestamp': '2025-04-14 09:18:37', 'Open': np.float64(388.81), 'High': np.float64(389.46), 'Low': np.float64(387.46), 'Close': np.float64(387.99), 'Volume': 2510488}}
Sent: {'ticker': 'AMD', 'data': {'timestamp': '2025-04-14 09:18:37', 'Open': np.float64(93.43), 'High': np.float64(93.58), 'Low': np.float64(92.89), 'Close': np.float64(93.3), 'Volume': 3709673}}
Sent: {'ticker': 'AMZN', 'data': {'timestamp': '2025-04-14 09:18:37', 'Open': np.float64(184.86), 'High': np.float64(185.86), 'Low': np.f

KeyboardInterrupt: 